In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
torch.set_printoptions(linewidth=120)
from torchvision import datasets
import torchvision.transforms as transforms

In [ ]:
num_workers = 0
batch_size = 20
transform = transforms.ToTensor()

train_data = datasets.MNIST(root='data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers)

device = torch.device("cuda:0")

In [ ]:
train_loader.dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1) 
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0)
        self.dropout2 = nn.Dropout2d(0)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

model = Net()
model.to(device)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout2d(p=0, inplace=False)
  (dropout2): Dropout2d(p=0, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
n_epochs = 1
model.train()
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
confidence = list(0. for i in range(10)) 

for epoch in range(n_epochs):
    train_loss = 0.0
    
    
    for data, target in train_loader:
        data=data.to(device)
        target=target.to(device)
        optimizer.zero_grad()
        output = model(data)
        a, pred = torch.max(output, 1)
        correct = np.squeeze(pred.eq(target.data.view_as(pred)))
        count=0
        for i in range(batch_size):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1
            if a[count].item()<=confidence[label]:
              confidence[label]=a[count].item()
            count+=1


        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
        
    train_loss = train_loss/len(train_loader.dataset)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch+1, 
        train_loss
        ))
model.eval()

Epoch: 1 	Training Loss: 0.150676


Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout2d(p=0, inplace=False)
  (dropout2): Dropout2d(p=0, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [ ]:
confidence

[-2.215944766998291,
 -2.2253644466400146,
 -2.2421669960021973,
 -2.222172498703003,
 -2.231606960296631,
 -2.228450298309326,
 -2.2422738075256348,
 -2.2339107990264893,
 -2.2306504249572754,
 -2.2370591163635254]

In [ ]:
test_loss = 0.0
class_correct = list(0. for i in range(11))
class_total = list(0. for i in range(11))
model.eval() 

for data, target in test_loader:
    data=data.to(device)
    target=target.to(device)
    output = model(data)
    loss = criterion(output, target)
    test_loss += loss.item()*data.size(0)
    a, pred = torch.max(output, 1)
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.063188

Test Accuracy of     0: 99% (971/980)
Test Accuracy of     1: 99% (1130/1135)
Test Accuracy of     2: 98% (1015/1032)
Test Accuracy of     3: 98% (996/1010)
Test Accuracy of     4: 97% (961/982)
Test Accuracy of     5: 99% (888/892)
Test Accuracy of     6: 96% (920/958)
Test Accuracy of     7: 96% (989/1028)
Test Accuracy of     8: 96% (941/974)
Test Accuracy of     9: 97% (988/1009)

Test Accuracy (Overall): 97% (9799/10000)


In [ ]:
class_correct = list(0. for i in range(11))
class_total = list(0. for i in range(11))
for data, target in test_loader:
    data=data.to(device)
    target=target.to(device)
    output = model(data)
    #print(target)
    a, pred = torch.max(output, 1)
    print(a)
    print(target)
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    count=0
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1
        if a[count].item()<=min(confidence):
          class_correct[10] += correct[i].item()
          class_total[10] += 1
        count+=1
    
    break
#print(pred)
#print(target)
print(class_correct)
print(confidence)


tensor([-2.2173e-05, -3.8147e-06, -1.6925e-03, -2.9563e-05, -2.5868e-05, -1.4808e-03, -1.5727e-03, -3.5926e-03,
        -1.2164e-01, -3.0401e-03, -1.1921e-07, -1.0014e-05, -1.5974e-05, -5.9605e-07, -5.1139e-05, -3.0167e-04,
        -9.5840e-05, -3.6007e-04, -9.5398e-01, -2.1458e-06], device='cuda:0', grad_fn=<MaxBackward0>)
tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4], device='cuda:0')
[3.0, 3.0, 1.0, 1.0, 3.0, 2.0, 1.0, 2.0, 0.0, 4.0, 0.0]
[-1.4688225984573364, -1.0965884923934937, -1.2206205129623413, -1.269418716430664, -1.0011956691741943, -1.2658755779266357, -0.9875769019126892, -1.0830365419387817, -1.156612753868103, -1.176843285560608]
